In [1]:
import os
import warnings
import re
import random
import math
import pandas as pd
import numpy as np
import scipy
import sklearn

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from pylab import rcParams

In [4]:
feeds_df = pd.read_csv(os.getcwd() + '/Model_Output/feeds_df.csv')
feeds_df['createdAt'] = pd.to_datetime(feeds_df['createdAt'])
feeds_df['updatedAt'] = pd.to_datetime(feeds_df['updatedAt'])
feeds_df = feeds_df[~feeds_df['text'].isna()]
print(feeds_df.shape)
feeds_df.head()

(3519, 12)


,contentId,anonymous,authorId,createdAt,isActive,isDelete,points,text,type,updatedAt,resource_videoUrl,resource_image
0,5ea9cf97835077004dca1290,False,5ea6d59f598ec5004073b192,2020-04-29 19:03:51.089,False,False,0.000000,Hello teachers and educators!We have come up w...,POST,2020-04-30 02:56:06.516,[],[{'url': 'https://res.cloudinary.com/koolkanya...
1,5ea9cfc56ce29f002dcd6832,False,5e9d471fb55aa50028ebcfbc,2020-04-29 19:04:37.991,True,False,0.431818,We never lose. We only learn. During this lock...,POST,2020-04-30 15:49:27.459,[],[]
2,5ea9df6fc8252f0022a16680,False,5ea88300cd36e60022bf2fee,2020-04-29 20:11:27.795,True,False,0.750000,Hello kanyas! Here is my 2nd picture. This is ...,POST,2020-04-30 13:35:08.028,[],[{'url': 'https://res.cloudinary.com/koolkanya...
3,5ea9f262fbd4eb0028b9cc5c,False,5e809291b0854d0031972a19,2020-04-29 21:32:18.212,False,False,0.000000,"When she said, ‘She is on her periods.’As we w...",POST,2020-04-30 02:59:54.397,[],[{'url': 'https://res.cloudinary.com/koolkanya...
4,5eaa360ed493bf0033cebbbc,False,5ea79f27905def0034f7a8ee,2020-04-30 02:21:02.237,True,False,0.500000,"WOMEN AT 20'S - 30'S Health is wealth, we w...",POST,2020-04-30 13:16:48.752,[],[]


# Other Datasets

In [5]:
#interactions_train_indexed_df = pd.read_csv(os.getcwd() + '/Model_Output/interactions_train_indexed_df.csv')
#interactions_train_indexed_df.head()

In [6]:
interaction_total_indexed_df = pd.read_csv(os.getcwd() + '/Model_Output/interaction_total_indexed_df.csv')
interaction_total_indexed_df.head()

,personId,contentId,eventStrength
0,5df38de0b2694d0016bdb71f,5df7222ed092c70016203257,2.584963
1,5df38de0b2694d0016bdb71f,5df72230d092c7001620325d,2.584963
2,5df38de0b2694d0016bdb71f,5df72233d092c70016203263,2.584963
3,5df38de0b2694d0016bdb71f,5df72235d092c7001620326a,2.584963
4,5df38de0b2694d0016bdb71f,5df72238d092c70016203271,2.584963


In [7]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    if(((pd.Series(person_id).isin(pd.Series(interactions_df.index.unique().to_list()))) == True)[0]):
        interacted_items = interactions_df.loc[person_id]['contentId']
        return(set(interacted_items if type(interacted_items) == pd.Series else [interacted_items]))
    else:
        interacted_items = set()
        return(interacted_items)
    

In [8]:
recency_df = feeds_df[['contentId', 'createdAt']].copy()
recency_df['time_since_post'] = recency_df['createdAt'].max() - recency_df['createdAt']
recency_df = recency_df.sort_values(['time_since_post'], ascending=True)
recency_df['recency_rank'] = list(range(recency_df.shape[0]))
print(recency_df.shape)
recency_df.head()

(3116, 4)


,contentId,createdAt,time_since_post,recency_rank
1088,5ed5e4576c97df0022ac67ac,2020-06-02 05:32:07.400,00:00:00,0
1136,5ed5d9156c97df0022ac6769,2020-06-02 04:44:05.571,00:48:01.829000,1
985,5ed5d1eb50110f0028003392,2020-06-02 04:13:31.749,01:18:35.651000,2
1042,5ed5c76b50110f002800336c,2020-06-02 03:28:43.329,02:03:24.071000,3
1107,5ed5c64150110f0028003365,2020-06-02 03:23:45.483,02:08:21.917000,4


In [9]:
def recency_of_recommendation_func(all_recommendations, person_id):
    recommendation_with_recency = pd.merge(all_recommendations, recency_df[['contentId', 'createdAt', 'time_since_post', 'recency_rank']], how = 'left', on = 'contentId')
    recommendation_with_recency = recommendation_with_recency[recommendation_with_recency['recency_rank']<=800]
    return(recommendation_with_recency)

# Popularity Model

In [10]:
item_popularity_df = pd.read_csv(os.getcwd() + '/Model_Output/item_popularity_df.csv')
item_popularity_df.head()

,contentId,eventStrength
0,5eb4f43d63c2df0022d975b8,141.070747
1,5ebb8db54b04f6003dd86570,140.049821
2,5eccf85617d7ed0024840e9d,132.190416
3,5e782cd3ea7e440022515910,117.255353
4,5ea2c5e5d4abf40029d88e2f,110.661180


In [11]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)] \
                               .sort_values('eventStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['eventStrength', 'contentId']]
        
        #recommendations_df = recency_of_recommendation_func(recommendations_df, user_id)


        return recommendations_df
    
popularity_model = PopularityRecommender(item_popularity_df, feeds_df)

In [12]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
u_id = '5df76af011788b0016813414'
user_rec = popularity_model.recommend_items(u_id, items_to_ignore=get_items_interacted(u_id, 
                                         interaction_total_indexed_df), 
                                         topn=100)
user_rec = pd.merge(user_rec, feeds_df[['contentId', 'text']], how = 'left', on = 'contentId')
user_rec

,contentId,eventStrength,text
0,5eb4f43d63c2df0022d975b8,141.070747,MOTHER'S DAY CONTEST - THIRD DAY QUESTION! In ...
1,5ebb8db54b04f6003dd86570,140.049821,Each Wednesday is for you to talk about and pr...
2,5eccf85617d7ed0024840e9d,132.190416,"Kool Kanya presents GetWork, our first communi..."
3,5e782cd3ea7e440022515910,117.255353,A big warm welcome to you all! We were so eage...
4,5ea2c5e5d4abf40029d88e2f,110.661180,Share your favourite book with the Community. ...
5,5eb64ff240ca75004c486c14,81.650041,MOTHER'S DAY CONTEST - FOURTH DAY QUESTION! In...
6,5ecf500d430327002815ba36,78.477053,"Kool Kanya presents GetWork, our first communi..."
7,5eb39dc7854ebb0022622aa0,73.061326,MOTHER'S DAY CONTEST - SECOND DAY QUESTION! In...
8,5eb78f0117e19d00491830ab,65.920599,MOTHER'S DAY CONTEST - FIFTH DAY QUESTION! S...
9,5eb23edc7056fb002fab0c65,65.753619,MOTHER'S DAY CONTEST - FIRST DAY QUESTION! I...


# Content-Based Filtering model

In [14]:
item_ids = feeds_df['contentId'].tolist()

In [15]:
import pickle

with open(os.getcwd() + '/Model_Output/tfidf_matrix.pkl', 'rb') as f:
    tfidf_matrix = pickle.load(f)
tfidf_matrix

<3116x1722 sparse matrix of type '<class 'numpy.float64'>'
	with 65874 stored elements in Compressed Sparse Row format>

In [16]:
import pickle

In [17]:
with open(os.getcwd() + '/Model_Output/user_profiles_all.pkl', 'rb') as f:
    user_profiles = pickle.load(f)
user_profiles

{'5df38de0b2694d0016bdb71f': array([[0.05259468, 0.0506922 , 0.00569125, ..., 0.00202336, 0.00432202,
         0.07057204]]),
 '5df431157e4ea600161b8e13': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '5df454297e4ea600161b8e25': array([[0.        , 0.        , 0.        , ..., 0.        , 0.02043089,
         0.01779454]]),
 '5df454d27e4ea600161b8e2a': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '5df4561e7e4ea600161b8e3e': array([[0.        , 0.01156354, 0.        , ..., 0.01607289, 0.01381358,
         0.02087149]]),
 '5df456267e4ea600161b8e43': array([[0.        , 0.0491128 , 0.06025241, ..., 0.        , 0.        ,
         0.        ]]),
 '5df456497e4ea600161b8e49': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '5df457557e4ea600161b8e5c': array([[0., 0., 0., ..., 0., 0., 0.]]),
 '5df4577c7e4ea600161b8e60': array([[0.        , 0.02517288, 0.        , ..., 0.        , 0.02874816,
         0.        ]]),
 '5df4579f7e4ea600161b8e68': array([[0.        , 0.04605051, 0.        , ..., 0.        , 0.  

In [18]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId']]
        
        recommendations_df = recency_of_recommendation_func(recommendations_df, user_id)
        
        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(feeds_df)

In [19]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [20]:
u_id = '5e85f9294841b100281d4709'
user_rec = content_based_recommender_model.recommend_items(u_id, items_to_ignore=get_items_interacted(u_id, 
                                         interaction_total_indexed_df), 
                                         topn=100)
user_rec = pd.merge(user_rec, feeds_df[['contentId', 'text']], how = 'left', on = 'contentId')
user_rec

,contentId,recStrength,createdAt,time_since_post,recency_rank,text
0,5ec7dcfdb85cf30022710734,0.291956,2020-05-22 14:09:01.747,10 days 15:23:05.653000,460,"Uncertain, Unplanned, Unprofessional- a Botany..."
1,5ebbc2bc5f3d60002c21ba48,0.287195,2020-05-13 09:49:48.181,19 days 19:42:19.219000,715,"Hi ladies, sorry have not been in active touch..."
2,5ec5f49d69b6bc002335ed42,0.262044,2020-05-21 03:25:17.920,12 days 02:06:49.480000,496,A series of unfortunate events. That's how I w...
3,5ec61ade3f0c290022fcb9f3,0.258813,2020-05-21 06:08:30.068,11 days 23:23:37.332000,491,#MyCareerstory My career story is interestin...
4,5ece00a0b3d9140022f22ca5,0.258726,2020-05-27 05:54:40.472,5 days 23:37:26.928000,324,"#mycareerstory Hello, Kool Kanya's I am Bha..."
5,5ec3fb1990177000284c27c9,0.247361,2020-05-19 15:28:25.334,13 days 14:03:42.066000,527,My thoughts on Getting Laid Off #MyCareerStor...
6,5ecf506f430327002815ba3b,0.247175,2020-05-28 05:47:27.610,4 days 23:44:39.790000,257,#MyCareerStoryI was working as an Operations M...
7,5ec3e87c778b540022d04f9a,0.235429,2020-05-19 14:09:00.281,13 days 15:23:07.119000,529,Hi Yesterday I had #onlinecounseling with my f...
8,5ecb875b1ca3c6002422cd2d,0.232802,2020-05-25 08:52:43.378,7 days 20:39:24.022000,405,"This is a long-awaited confession, a story abo..."
9,5eb9a48672f1ee0027d10313,0.230351,2020-05-11 19:16:22.494,21 days 10:15:44.906000,794,"Hello Everyone, I hope you all are staying saf..."


# Collaborative Filtering model

### Matrix Factorization

In [21]:
import pickle

with open(os.getcwd() + '/Model_Output/cf_preds_df_all.pkl', 'rb') as f:
    cf_preds_df = pickle.load(f)
cf_preds_df.shape

(3116, 2732)

In [22]:
cf_preds_df.head(2)

,5df38de0b2694d0016bdb71f,5df431157e4ea600161b8e13,5df454297e4ea600161b8e25,5df454d27e4ea600161b8e2a,5df4561e7e4ea600161b8e3e,5df456267e4ea600161b8e43,5df456497e4ea600161b8e49,5df457557e4ea600161b8e5c,5df4577c7e4ea600161b8e60,5df4579f7e4ea600161b8e68,...,5ed5ab7c6c97df0022ac66c7,5ed5bb7350110f0028003341,5ed5c1146c97df0022ac6715,5ed5c49e50110f0028003359,5ed5c64d50110f0028003366,5ed5cb3d50110f0028003374,5ed5da8b50110f00280033b6,5ed5dee150110f00280033db,5ed5e20450110f00280033f9,5ed5e2796c97df0022ac679c
contentId,,,,,,,,,,,,,,,,,,,,,
5df454f37e4ea600161b8e2e,0.133965,0.134046,0.134602,0.132916,0.138182,0.135106,0.132906,0.133562,0.133688,0.141989,...,0.132879,0.132877,0.132862,0.132879,0.132879,0.132864,0.132879,0.132859,0.132879,0.132864
5df4556a7e4ea600161b8e32,0.132633,0.133808,0.134215,0.132895,0.136549,0.134019,0.132877,0.133501,0.132833,0.138934,...,0.132879,0.132879,0.132934,0.132879,0.132879,0.132934,0.132879,0.132946,0.132879,0.132934


In [23]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['contentId'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId']]

        recommendations_df = recency_of_recommendation_func(recommendations_df, user_id)
        return recommendations_df
    
cf_recommender_model = CFRecommender(cf_preds_df, feeds_df)

In [24]:
u_id = '5df72acdd092c70016203ac5'
user_rec = cf_recommender_model.recommend_items(u_id, items_to_ignore=get_items_interacted(u_id, 
                                         interaction_total_indexed_df), 
                                         topn=1000)
user_rec = pd.merge(user_rec, feeds_df[['contentId', 'text']], how = 'left', on = 'contentId')
user_rec

,contentId,recStrength,createdAt,time_since_post,recency_rank,text
0,5eccf577b3d9140022f22598,0.206125,2020-05-26 10:54:47.927,6 days 18:37:19.473000,361,How can a person who has been a full time empl...
1,5ece14da430327002815b23c,0.204426,2020-05-27 07:20:58.854,5 days 22:11:08.546000,314,Fresher content writer: This agency based in C...
2,5eccb6421b8d780022260b18,0.198392,2020-05-26 06:25:07.000,6 days 23:07:00.400000,372,"Ladies, we are back with another Zoom session ..."
3,5ece4b97462d280022453862,0.198392,2020-05-27 11:14:31.323,5 days 18:17:36.077000,301,Hey kool kanyas.
4,5ed0d767b99db90024a7256d,0.194658,2020-05-29 09:35:35.829,3 days 19:56:31.571000,170,"Freelancers, are you read to GetWork with us o..."
5,5ec621903f0c290022fcba2c,0.193147,2020-05-21 06:37:04.893,11 days 22:55:02.507000,488,#MyCareerStory From not doing a typical MBA to...
6,5ece36b5462d28002245379a,0.192183,2020-05-27 09:45:25.133,5 days 19:46:42.267000,305,Hey I'm an watercolor artist Is there anything...
7,5ebba10c4b04f6003dd86590,0.187916,2020-05-13 07:26:04.124,19 days 22:06:03.276000,721,Hello everyone! I am new to this platform. My ...
8,5ec01b5d7d2588002321b534,0.181060,2020-05-16 16:57:01.852,16 days 12:35:05.548000,613,Hello everyone. I just want to share my feelin...
9,5ec3e5bc778b540022d04f94,0.180461,2020-05-19 13:57:16.882,13 days 15:34:50.518000,530,So I just bagged an offer for a Sales Associat...


### Hybrid Recommendation 

In [25]:
class HybridRecommender:
    
    MODEL_NAME = 'Hybrid'
    
    def __init__(self, cb_rec_model, cf_rec_model, items_df, cb_ensemble_weight=1.0, cf_ensemble_weight=1.0):
        self.cb_rec_model = cb_rec_model
        self.cf_rec_model = cf_rec_model
        self.cb_ensemble_weight = cb_ensemble_weight
        self.cf_ensemble_weight = cf_ensemble_weight
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        #Getting the top-1000 Content-based filtering recommendations
        cb_recs_df = self.cb_rec_model.recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose,
                                                           topn=1000).rename(columns={'recStrength': 'recStrengthCB'})
        cb_recs_df = cb_recs_df.drop(['createdAt', 'time_since_post', 'recency_rank'], axis=1)
        
        #Getting the top-1000 Collaborative filtering recommendations
        cf_recs_df = self.cf_rec_model.recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose, 
                                                           topn=1000).rename(columns={'recStrength': 'recStrengthCF'})
        cf_recs_df = cf_recs_df.drop(['createdAt', 'time_since_post', 'recency_rank'], axis=1)
        
        #Combining the results by contentId
        recs_df = cb_recs_df.merge(cf_recs_df,
                                   how = 'outer', 
                                   left_on = 'contentId', 
                                   right_on = 'contentId').fillna(0.0)
        
        #Computing a hybrid recommendation score based on CF and CB scores
        #recs_df['recStrengthHybrid'] = recs_df['recStrengthCB'] * recs_df['recStrengthCF'] 
        recs_df['recStrengthHybrid'] = (recs_df['recStrengthCB'] * self.cb_ensemble_weight) \
                                     + (recs_df['recStrengthCF'] * self.cf_ensemble_weight)
        
        #Sorting recommendations by hybrid score
        recommendations_df = recs_df.sort_values('recStrengthHybrid', ascending=False).head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrengthHybrid', 'contentId']]

        recommendations_df = recency_of_recommendation_func(recommendations_df, user_id)
        return recommendations_df
    
hybrid_recommender_model = HybridRecommender(content_based_recommender_model, cf_recommender_model, feeds_df,
                                             cb_ensemble_weight=1.0, cf_ensemble_weight=50.0)


In [26]:
u_id = '5df72acdd092c70016203ac5'
user_rec = hybrid_recommender_model.recommend_items(u_id, items_to_ignore=get_items_interacted(u_id, 
                                         interaction_total_indexed_df), 
                                         topn=100)
user_rec = pd.merge(user_rec, feeds_df[['contentId', 'text']], how = 'left', on = 'contentId')
user_rec

,contentId,recStrengthCB,recStrengthCF,recStrengthHybrid,createdAt,time_since_post,recency_rank,text
0,5ece14da430327002815b23c,0.149756,0.204426,10.371054,2020-05-27 07:20:58.854,5 days 22:11:08.546000,314,Fresher content writer: This agency based in C...
1,5eccf577b3d9140022f22598,0.000000,0.206125,10.306233,2020-05-26 10:54:47.927,6 days 18:37:19.473000,361,How can a person who has been a full time empl...
2,5ece4b97462d280022453862,0.225215,0.198392,10.144800,2020-05-27 11:14:31.323,5 days 18:17:36.077000,301,Hey kool kanyas.
3,5eccb6421b8d780022260b18,0.162298,0.198392,10.081882,2020-05-26 06:25:07.000,6 days 23:07:00.400000,372,"Ladies, we are back with another Zoom session ..."
4,5ec621903f0c290022fcba2c,0.184236,0.193147,9.841564,2020-05-21 06:37:04.893,11 days 22:55:02.507000,488,#MyCareerStory From not doing a typical MBA to...
5,5ed0d767b99db90024a7256d,0.000000,0.194658,9.732885,2020-05-29 09:35:35.829,3 days 19:56:31.571000,170,"Freelancers, are you read to GetWork with us o..."
6,5ece36b5462d28002245379a,0.000000,0.192183,9.609126,2020-05-27 09:45:25.133,5 days 19:46:42.267000,305,Hey I'm an watercolor artist Is there anything...
7,5ebba10c4b04f6003dd86590,0.177813,0.187916,9.573620,2020-05-13 07:26:04.124,19 days 22:06:03.276000,721,Hello everyone! I am new to this platform. My ...
8,5ec01b5d7d2588002321b534,0.208804,0.181060,9.261787,2020-05-16 16:57:01.852,16 days 12:35:05.548000,613,Hello everyone. I just want to share my feelin...
9,5ebce33b9acd57002817f9ae,0.178662,0.180162,9.186760,2020-05-14 06:20:43.888,18 days 23:11:23.512000,687,Learn how to promote your business on Google i...
